# Using rail_fsps to generate galaxy rest-frame spectral energy distributions and  compute apparent magnitudes

author: Luca Tortorelli, Josue de Santiago, Eric Charles
    
last run successfully: Aug 2nd, 2023
    
This notebook demonstrates how to use rail_fsps to generate galaxy rest-frame spectral energy distributions (SEDs) with FSPS and how to compute apparent magnitudes from them.

In order to run this notebook you need to have FSPS and Python-FSPS installed. The easiest way to do this is the following (first line applies only in case you already installed it via pip):

    pip uninstall fsps
    git clone --recursive https://github.com/dfm/python-fsps.git
    cd python-fsps
    python -m pip install .
    export SPS_HOME=$(pwd)/src/fsps/libfsps

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from rail.creation.engines.fsps_sed_modeler import *
from rail.utils.path_utils import find_rail_file

from rail import interactive as ri

We'll start by setting up the paths we'll need to access data from.

In [ ]:
input_file = find_rail_file(
    "examples_data/creation_data/data/fsps_default_data/input_galaxy_properties_fsps.hdf5"
)

We generate some mock input data for the sed modeler class that we store in an hdf5 file.

In [ ]:
n_galaxies = 10

redshifts = np.linspace(0.1, 1, num=n_galaxies)
zmet = np.full(n_galaxies, 1, dtype=int)
stellar_metallicity = np.full(n_galaxies, 0.0)  # log10(Z/Zsun)
pmetals = np.full(n_galaxies, 2.0)
stellar_velocity_dispersion = np.full(n_galaxies, 100.0)
gas_ionization = np.full(n_galaxies, -1)
gas_metallicity = np.full(n_galaxies, 0.0)
tau = np.full(n_galaxies, 1.0)
const = np.full(n_galaxies, 0.0)
sf_start = np.full(n_galaxies, 0.0)
sf_trunc = np.full(n_galaxies, 0.0)
stellar_age = np.full(n_galaxies, 2.0)
fburst = np.full(n_galaxies, 0.0)
tburst = np.full(n_galaxies, 11.0)
sf_slope = np.full(n_galaxies, 0.0)
dust1_birth_cloud = np.full(n_galaxies, 0.1)
dust2_diffuse = np.full(n_galaxies, 0.1)
dust_index = np.full(n_galaxies, -0.7)
dust_calzetti_modifier = np.full(n_galaxies, -1.0)
mwr = np.full(n_galaxies, 3.1)
uvb = np.full(n_galaxies, 1.0)
wgp1 = np.full(n_galaxies, 1)
dust_gamma = np.full(n_galaxies, 0.01)
dust_umin = np.full(n_galaxies, 1.0)
dust_qpah = np.full(n_galaxies, 3.5)
f_agn = np.full(n_galaxies, 0.01)
tau_agn = np.full(n_galaxies, 10.0)

gal_t_table = np.linspace(0.05, 13.8, 100)  # age of the universe in Gyr
gal_sfr_table = np.random.uniform(0, 10, gal_t_table.size)  # SFR in Msun/yr
tabulated_sfh = np.full(
    (n_galaxies, 2, len(gal_sfr_table)), [gal_t_table, gal_sfr_table]
)

wave_lsf = np.linspace(3000, 10000, 2000)
sigma_lsf = np.full_like(wave_lsf, 0.5)
tabulated_lsf = np.full((n_galaxies, 2, len(wave_lsf)), [wave_lsf, sigma_lsf])

training_data = {
    "model": {
        "redshifts": redshifts,
        "zmet": zmet,
        "stellar_metallicity": stellar_metallicity,
        "pmetals": pmetals,
        "stellar_velocity_dispersion": stellar_velocity_dispersion,
        "gas_ionization": gas_ionization,
        "gas_metallicity": gas_metallicity,
        "tau": tau,
        "const": const,
        "sf_start": sf_start,
        "sf_trunc": sf_trunc,
        "stellar_age": stellar_age,
        "fburst": fburst,
        "tburst": tburst,
        "sf_slope": sf_slope,
        "dust1_birth_cloud": dust1_birth_cloud,
        "dust2_diffuse": dust2_diffuse,
        "dust_index": dust_index,
        "dust_calzetti_modifier": dust_calzetti_modifier,
        "mwr": mwr,
        "uvb": uvb,
        "wgp1": wgp1,
        "dust_gamma": dust_gamma,
        "dust_umin": dust_umin,
        "dust_qpah": dust_qpah,
        "f_agn": f_agn,
        "tau_agn": tau_agn,
        "tabulated_lsf": tabulated_lsf,
        "tabulated_sfh": tabulated_sfh,
    }
}

Let's create an FSPSSedModeler class object. The latter has a number of configuration parameters to set. All the parameters have default values. Therefore, if the user is not sure about a particular value for a certain parameter, the latter can be left at default. A short description of each parameter can be found in src/rail/creation/engines/fsps_sed_modeler.py. 

We run the FSPSSedModeler in sequential mode. Note that each galaxy spectrum takes a few seconds to generate, so it is advisable to proceed in this way only for a limited sample of objects or for testing the code.

In [ ]:
fspssedmodel = ri.creation.engines.fsps_sed_modeler.fsps_sed_modeler(
    input=training_data,
    chunk_size=10,
    hdf5_groupname="model",
    compute_vega_mags=False,
    vactoair_flag=False,
    zcontinuous=1,
    add_agb_dust_model=True,
    add_dust_emission=True,
    add_igm_absorption=True,
    add_neb_emission=True,
    add_neb_continuum=True,
    add_stellar_remnants=True,
    compute_light_ages=False,
    nebemlineinspec=True,
    smooth_velocity=True,
    smooth_lsf=False,
    cloudy_dust=False,
    agb_dust=1.0,
    tpagb_norm_type=2,
    dell=0.0,
    delt=0.0,
    redgb=1.0,
    agb=1.0,
    fcstar=1.0,
    sbss=0.0,
    fbhb=0.0,
    pagb=1.0,
    redshifts_key="redshifts",
    zmet_key="zmet",
    stellar_metallicities_key="stellar_metallicity",
    pmetals_key="pmetals",
    imf_type=1,
    imf_upper_limit=120.0,
    imf_lower_limit=0.08,
    imf1=1.3,
    imf2=2.3,
    imf3=2.3,
    vdmc=0.08,
    mdave=0.5,
    evtype=-1,
    use_wr_spectra=1,
    logt_wmb_hot=0.0,
    masscut=150.0,
    velocity_dispersions_key="stellar_velocity_dispersion",
    min_wavelength=3000,
    max_wavelength=10000,
    gas_ionizations_key="gas_ionization",
    gas_metallicities_key="gas_metallicity",
    igm_factor=1.0,
    sfh_type=3,
    tau_key="tau",
    const_key="const",
    sf_start_key="sf_start",
    sf_trunc_key="sf_trunc",
    stellar_ages_key="stellar_age",
    fburst_key="fburst",
    tburst_key="tburst",
    sf_slope_key="sf_slope",
    dust_type=2,
    dust_tesc=7.0,
    dust_birth_cloud_key="dust1_birth_cloud",
    dust_diffuse_key="dust2_diffuse",
    dust_clumps=-99,
    frac_nodust=0.0,
    frac_obrun=0.0,
    dust_index_key="dust_index",
    dust_powerlaw_modifier_key="dust_calzetti_modifier",
    mwr_key="mwr",
    uvb_key="uvb",
    wgp1_key="wgp1",
    wgp2=1,
    wgp3=1,
    dust_emission_gamma_key="dust_gamma",
    dust_emission_umin_key="dust_umin",
    dust_emission_qpah_key="dust_qpah",
    fraction_agn_bol_lum_key="f_agn",
    agn_torus_opt_depth_key="tau_agn",
    tabulated_sfh_key="tabulated_sfh",
    tabulated_lsf_key="tabulated_lsf",
    physical_units=False,
    restframe_wave_key="restframe_wavelengths",
    restframe_sed_key="restframe_seds",
)

Here we show the example where we provide tabulated star-formation histories to generate the final SED. In this case, FSPS outputs the emission per total stellar mass. We call the fit_model() function to generate the model SEDs.

In [ ]:
fspssedmodel["model"]

We plot the first spectrum to check that the SED generation worked correctly.

In [ ]:
redshifts = fspssedmodel["model"]["redshifts"]
restframe_seds = fspssedmodel["model"]["restframe_seds"]
restframe_wavelengths = fspssedmodel["model"]["restframe_wavelengths"]

In [ ]:
plt.clf()
plt.plot(restframe_wavelengths[0], restframe_seds[0], lw=2, color="black")
plt.xlim(3000, 10000)
plt.xlabel(r"wavelength [$\AA$]")
plt.ylabel(r"luminosity density [$\mathrm{Lsun \ Hz^{-1}}$]")
plt.show()